# IFRC Surge Alerts Analysis

This notebook fetches surge alerts from the IFRC GO API and analyzes deployment needs.

## 1. Import Required Libraries

In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
print('Environment variables loaded successfully!')

## 2. Define Functions

### 2.1 Get Authorization Headers

In [ ]:
def get_auth_headers():
    """
    Get authorization headers using token from environment variable.
    
    Returns:
        dict: Headers dictionary with Authorization token
    
    Raises:
        ValueError: If IFRC_API_TOKEN is not found in environment
    """
    auth_token = os.getenv('IFRC_API_TOKEN')
    
    if not auth_token:
        raise ValueError('IFRC_API_TOKEN not found in environment. Please check your .env file.')
    
    return {"Authorization": f"Token {auth_token}"}

### 2.2 Fetch Surge Alerts

In [ ]:
def fetch_surge_alerts(search):
    """
    Fetch all surge alerts from the GO API matching a search term.
    
    Args:
        search: Search term to filter surge alerts (e.g., 'Population Movement')
    
    Returns:
        list: List of dictionaries containing 'id' and 'deployment_needed' for each alert
    """
    all_surge_alerts = []
    offset = 0
    limit = 50  # Number of records per page
    
    headers = get_auth_headers()

    params = {
        'offset': offset,
        'limit': limit,
        'search': search
    }

    while True:
        response = requests.get('https://goadmin.ifrc.org/api/v2/surge_alert/',
                                headers=headers, params=params)
        
        # Check for successful response
        if response.status_code != 200:
            print(f"Error: API returned status code {response.status_code}")
            break
        
        data = response.json()
        surge_alerts = [{'id': alert['id'], 'deployment_needed': alert['deployment_needed']}
                        for alert in data.get('results', [])]

        # If there are no more surge alerts, exit the loop
        if not surge_alerts:
            break

        # Add surge alerts from the current page to the list
        all_surge_alerts.extend(surge_alerts)
        print(f"Fetched {len(surge_alerts)} alerts (total: {len(all_surge_alerts)})")

        # Move to the next page
        offset += limit
        params['offset'] = offset

    return all_surge_alerts

### 2.3 Count Deployments Needed

In [ ]:
def deployments_needed(surge_alerts):
    """
    Count surge alerts by deployment_needed status.
    
    Args:
        surge_alerts: List of surge alert dictionaries
    
    Returns:
        dict: Dictionary with 'labels' (unique deployment_needed values) 
              and 'data' (counts for each)
    """
    labels = []
    data_values = []

    for alert in surge_alerts:
        deployment_status = alert['deployment_needed']
        if deployment_status in labels:
            index = labels.index(deployment_status)
            data_values[index] += 1
        else:
            labels.append(deployment_status)
            data_values.append(1)

    chart_data = {
        'labels': labels,
        'data': data_values
    }
    return chart_data

## 3. Fetch Surge Alerts

In [ ]:
# Fetch surge alerts matching the search term
search_term = 'Population Movement'
surge_alerts = fetch_surge_alerts(search=search_term)

print(f"\nTotal surge alerts found for '{search_term}': {len(surge_alerts)}")

## 4. Analyze Deployment Needs

In [ ]:
# Count deployments by status
# FIX: Use the correct variable name 'surge_alerts' (was using undefined 'surge_alerts' vs 'no_surge_alerts')
deployments = deployments_needed(surge_alerts)

print("Deployment needs summary:")
print(deployments)

# Display in a more readable format
print("\nBreakdown:")
for label, count in zip(deployments['labels'], deployments['data']):
    status = "Yes" if label else "No"
    print(f"  Deployment needed = {status}: {count} alerts")

## 5. Summary

### Issues corrected from original code:

| Issue | Original | Fixed |
|-------|----------|-------|
| **Auth token** | Hardcoded string | Uses `.env` via `os.getenv()` |
| **Variable mismatch** | `no_surge_alerts` then used `surge_alerts` | Consistent variable name `surge_alerts` |
| **Error handling** | None | Added status code check |
| **Documentation** | No docstrings | Added docstrings to all functions |